In [1]:
import numpy as np
import pandas as pd
import itertools

import neo4j
import psycopg2

from geographiclib.geodesic import Geodesic

# Load data on chosen golf courses and their coordinates

In [2]:
courses_df = pd.read_csv("../data/chosen_courses.csv", usecols=['golf_course'])
geo_df = pd.read_csv("../data/distance_data.csv", usecols=['golf_course', 'golf_course_geo'])

courses_geo_df = pd.merge(courses_df, geo_df, on='golf_course', how='left')
courses_geo_df = courses_geo_df.drop_duplicates().reset_index(drop=True)
courses_geo_df

,golf_course,golf_course_geo
0,Trump Golf Links At Ferry Point,"(40.812471, -73.834446)"
1,Bulle Rock,"(39.5422532, -76.1328107)"
2,The Glen Club,"(42.090201, -87.82770579999999)"
3,Streamsong Resort (Red),"(27.6928094, -81.9344824)"
4,Reynolds Lake Oconee (Oconee),"(33.4398841, -83.1975021)"
5,Pelican Hill Golf Club (Ocean South),"(33.584694, -117.8403009)"
6,Lake Of Isles (North),"(41.4838118, -71.950374)"
7,Trump National Doral,"(25.8129673, -80.3387175)"


In [3]:
def get_lat_long(point):
    
    "Given string in (lat, long) format, extract list of floats in [lat, long] format"
    
    init_split = point.split('(')[1].split(',')
    lat = init_split[0]
    long = init_split[1].split(')')[0]
    
    return [float(lat), float(long)]

In [4]:
def calculate_distance(point_1, point_2):
    "Given two points in (latitude, longitude) format, calculate the distance between them in miles, using ellipsoid distnace for accuracy"
    
    geod = Geodesic.WGS84

    g = geod.Inverse(point_1[0], point_1[1], point_2[0], point_2[1])
    miles = g['s12'] / 1000 * 0.621371
    
    return miles

In [5]:
# Get all pairs of courses (excluding duplicates) and make df with corresponding coordinates

courses_pairs = [(c1, c2) for c1 in courses_df['golf_course'] for c2 in courses_df['golf_course'] if c1 != c2]

pairs_geo = []

for pair in courses_pairs:
    pairs_geo.append(
    {
    'Course_A': pair[0],
    'Course_B': pair[1],
    # Get corresponding coordinates as float
    'Geo_A': get_lat_long(courses_geo_df.loc[courses_geo_df['golf_course'] == pair[0], 'golf_course_geo'].iloc[0]),
    'Geo_B': get_lat_long(courses_geo_df.loc[courses_geo_df['golf_course'] == pair[1], 'golf_course_geo'].iloc[0])
    }
)
    
pairs_geo_df = pd.DataFrame(pairs_geo)
pairs_geo_df.head()

,Course_A,Course_B,Geo_A,Geo_B
0,Trump Golf Links At Ferry Point,Bulle Rock,"[40.812471, -73.834446]","[39.5422532, -76.1328107]"
1,Trump Golf Links At Ferry Point,The Glen Club,"[40.812471, -73.834446]","[42.090201, -87.82770579999999]"
2,Trump Golf Links At Ferry Point,Streamsong Resort (Red),"[40.812471, -73.834446]","[27.6928094, -81.9344824]"
3,Trump Golf Links At Ferry Point,Reynolds Lake Oconee (Oconee),"[40.812471, -73.834446]","[33.4398841, -83.1975021]"
4,Trump Golf Links At Ferry Point,Pelican Hill Golf Club (Ocean South),"[40.812471, -73.834446]","[33.584694, -117.8403009]"


# Calculate distance between each pair

In [6]:
# Get ellipsoid distance

pairs_geo_df['Distance_crow'] = pairs_geo_df.apply(lambda x: calculate_distance(x.Geo_A, x.Geo_B), axis=1)
pairs_geo_df.head()

,Course_A,Course_B,Geo_A,Geo_B,Distance_crow
0,Trump Golf Links At Ferry Point,Bulle Rock,"[40.812471, -73.834446]","[39.5422532, -76.1328107]",149.912644
1,Trump Golf Links At Ferry Point,The Glen Club,"[40.812471, -73.834446]","[42.090201, -87.82770579999999]",731.036255
2,Trump Golf Links At Ferry Point,Streamsong Resort (Red),"[40.812471, -73.834446]","[27.6928094, -81.9344824]",1015.056069
3,Trump Golf Links At Ferry Point,Reynolds Lake Oconee (Oconee),"[40.812471, -73.834446]","[33.4398841, -83.1975021]",724.264110
4,Trump Golf Links At Ferry Point,Pelican Hill Golf Club (Ocean South),"[40.812471, -73.834446]","[33.584694, -117.8403009]",2451.735798


# Neo4j functions from labs

In [7]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [8]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [9]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")
    
    display(df)

In [10]:
def my_neo4j_create_golf_course_node(golf_course_name):
    "create a node with label Golf_Course"
    
    query = """
    
    CREATE (:Golf_Course {name: $golf_course_name})
    
    """
    
    session.run(query, golf_course_name=golf_course_name)

In [11]:
def my_neo4j_create_relationship_one_way(from_course, to_course, weight):
    "create a relationship one way between  two golf courses with a weight"
    
    query = """
    
    MATCH (from:Golf_Course), 
          (to:Golf_Course)
    WHERE from.name = $from_course and to.name = $to_course
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_course=from_course, to_course=to_course, weight=weight)

In [12]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [13]:
cursor = connection.cursor()

# Create graph of golf courses weighted by distance

In [14]:
# Neo4j setup
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))
session = driver.session(database="neo4j")

my_neo4j_wipe_out_database()

In [15]:
# Check that graph is empty

my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 0
  Relationships: 0
-------------------------


,node_name_1,node_1_labels,relationship_type,node_name_2,node_2_labels


In [16]:
### Create graph of golf courses weighted by distance

# Keep track of nodes to avoid duplicates
nodes = []

for index, row in pairs_geo_df.iterrows():
    
    # Nodes for each golf course
    if row['Course_A'] not in nodes:
        my_neo4j_create_golf_course_node(row['Course_A'])
        nodes.append(row['Course_A'])
    if row['Course_B'] not in nodes:
        my_neo4j_create_golf_course_node(row['Course_B'])
        nodes.append(row['Course_B'])
    
    # Use distance as weight
    weight = row['Distance_crow']
    
    # Create one-way relationships between golf courses
    # Graph will have two-way relationships due to both directions being captured
    my_neo4j_create_relationship_one_way(row['Course_A'], row['Course_B'], weight)

In [17]:
## View nodes and relationships

my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 8
  Relationships: 56
-------------------------


,node_name_1,node_1_labels,relationship_type,node_name_2,node_2_labels
0,Bulle Rock,[Golf_Course],LINK,Lake Of Isles (North),[Golf_Course]
1,Bulle Rock,[Golf_Course],LINK,Pelican Hill Golf Club (Ocean South),[Golf_Course]
2,Bulle Rock,[Golf_Course],LINK,Reynolds Lake Oconee (Oconee),[Golf_Course]
3,Bulle Rock,[Golf_Course],LINK,Streamsong Resort (Red),[Golf_Course]
4,Bulle Rock,[Golf_Course],LINK,The Glen Club,[Golf_Course]
5,Bulle Rock,[Golf_Course],LINK,Trump Golf Links At Ferry Point,[Golf_Course]
6,Bulle Rock,[Golf_Course],LINK,Trump National Doral,[Golf_Course]
7,Lake Of Isles (North),[Golf_Course],LINK,Bulle Rock,[Golf_Course]
8,Lake Of Isles (North),[Golf_Course],LINK,Pelican Hill Golf Club (Ocean South),[Golf_Course]
9,Lake Of Isles (North),[Golf_Course],LINK,Reynolds Lake Oconee (Oconee),[Golf_Course]


# Minimum Spanning Tree

In [18]:
def my_neo4j_wipe_out_mst_relationships():
    "wipe out mst relationships"
    
    query = "match (node)-[relationship:MST]->() delete relationship"
    session.run(query)

In [19]:
my_neo4j_wipe_out_mst_relationships()

In [20]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = """

CALL gds.graph.project('ds_graph', 'Golf_Course', 
                        {
                            LINK: {
                                properties: 'weight',
                                orientation: 'UNDIRECTED'
                            }
                        }
                       )

"""

session.run(query)

In [21]:
query = """

MATCH (n:Golf_Course {name: $source})
CALL gds.beta.spanningTree.write('ds_graph',
                                          {sourceNode: id(n),
                                           relationshipWeightProperty: 'weight',
                                           writeProperty: 'writeCost',
                                           writeRelationshipType: 'MST'
                                          }
                                         )
YIELD preProcessingMillis, computeMillis, writeMillis, effectiveNodeCount
RETURN preProcessingMillis, computeMillis, writeMillis, effectiveNodeCount;

"""

source = courses_df['golf_course'][0] # start at highest weighted golf course

my_neo4j_run_query_pandas(query, source=source)

,preProcessingMillis,computeMillis,writeMillis,effectiveNodeCount
0,0,3,9,8


In [22]:
query = """

MATCH path = (n:Golf_Course {name: $source})-[:MST*]-()
WITH relationships(path) AS rels
UNWIND rels AS rel
WITH DISTINCT rel AS rel
RETURN startNode(rel).name AS source, endNode(rel).name AS destination, rel.writeCost AS distance

"""

source = courses_df['golf_course'][0]  # start at highest weighted golf course

min_span_tree = my_neo4j_run_query_pandas(query, source=source)
print(min_span_tree)

                            source                           destination  \
0  Trump Golf Links At Ferry Point                            Bulle Rock   
1                       Bulle Rock                         The Glen Club   
2                    The Glen Club  Pelican Hill Golf Club (Ocean South)   
3                       Bulle Rock         Reynolds Lake Oconee (Oconee)   
4    Reynolds Lake Oconee (Oconee)               Streamsong Resort (Red)   
5          Streamsong Resort (Red)                  Trump National Doral   
6  Trump Golf Links At Ferry Point                 Lake Of Isles (North)   

      distance  
0   149.912644  
1   637.210292  
2  1732.475319  
3   575.590317  
4   402.980517  
5   162.716933  
6   108.645461  


In [23]:
# Export Minimum Spanning Tree to CSV

min_span_tree.to_csv('../data/min_span_tree.csv', index=False)